![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FGenerate&file=Vertex+AI+Gemini+API.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Vertex%20AI%20Gemini%20API.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FGenerate%2FVertex%2520AI%2520Gemini%2520API.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Vertex%20AI%20Gemini%20API.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Generate/Vertex%20AI%20Gemini%20API.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Vertex AI Gemini API

Gemini is a family of foundation models that you can test, deploy, and customize using Vertex AI.  These models are hosted as an API that you can easily test: [with code](https://cloud.google.com/vertex-ai/generative-ai/docs/start/quickstarts/quickstart-multimodal), or interactively [with Vertex AI Studio](https://cloud.google.com/vertex-ai/generative-ai/docs/start/quickstarts/quickstart).

[Gemini models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models) are part of the family of [Google models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models) you will find in the [Vertex AI Model Garden](https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models).  This also includes [Partner Models](https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/use-partner-models), like Anthropic Claude and Llama, as well as [Open models](https://cloud.google.com/vertex-ai/generative-ai/docs/open-models/use-open-models) like Gemma and Hex-LLM.

The focus of this workflow is a deep dive into all of the many features of the [Gemini API](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference).

**References:**


---
**depricate this content once complete**

This notebook will hold an expanded overview of the Gemini API that is current found at the top of this workflow:

Combining content from:
- [Getting Started - Vertex AI GenAI Python Client](../Getting%20Started%20-%20Vertex%20AI%20GenAI%20Python%20Client.ipynb)
- [Python Asynchronous API Calls](../Python%20Asynchronous%20API%20Calls.ipynb)

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [11]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.69.0'),
    ('numpy', 'numpy'),
    ('matplotlib', 'matplotlib')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'gemini-api'

packages:

In [15]:
import io, base64

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Markdown

from google.cloud import aiplatform
import vertexai
import vertexai.generative_models # for Gemini Models

In [9]:
aiplatform.__version__

'1.69.0'

clients:

In [10]:
vertexai.init(project = PROJECT_ID, location = REGION)

---
## Gemini Models

Select one of the [supported Gemini models](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported-models) and read more about the characteristics of each [here](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).


### Setup Model

Here the [Gemini 1.5 Flash model with version 002](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-1.5-flash) is selected. It has these characteristics (to name a few):
- Max Input Tokens: 1,048,576
- Max Output Tokens: 8,192
- Max image:
    - raw size 20MB
    - base64 encoded size 7MB
    - number per prompt 3000
- Max video:
    - length 1 hour
    - number per prompt 10
- Max audio:
    - length 8.4 hours
    - number per prompt 1
- Max PDF:
    - size 30 MB
- 102 [Languages](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#languages-gemini) for understanding and responding


In [13]:
gemini = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-002")

Todo:
- call
- generation config call: break down by parameter
- system instruction
- multiple calls - async
- multimodal: image, video, audio, pdf
- chat
- section on limits, provision and dynamic (002 and higher)
- handling errors in responses
- caching
- safety config
- batch
- labels
- 